## DDPM 기초 구현
By https://github.com/CodingVillainKor/SimpleDeepLearning/blob/main/DDPM_notebook.ipynb + https://www.youtube.com/watch?v=svSQhYGKk0Q

### 구현해야될 항목
  - 각종 변수들 ($\alpha, \tilde\alpha, \mu, ...$)
  - 훈련 코드
    1. **repeat**:
       - \($ \mathbf{x}_0 \sim q(\mathbf{x}_0) $\)  (데이터 분포에서 샘플링)
       - \($ t \sim \text{Uniform}(\{1, \dots, T\})$ \)  (랜덤한 시간 스텝 샘플링)
       - \($ \epsilon \sim \mathcal{N}(\mathbf{0}, \mathbf{I})$ \)  (정규분포에서 노이즈 샘플링)
       - Gradient descent step on:
    
       - $
         \nabla_\theta \left\| \epsilon - \epsilon_\theta \left( \sqrt{\bar{\alpha}_t} \mathbf{x}_0 + \sqrt{1 - \bar{\alpha}_t} \epsilon, t \right) \right\|^2
         $
    
    2. **until converged**

  - 샘플링 코드
    1. **Initialize**:  
       - \($ \mathbf{x}_T \sim \mathcal{N}(\mathbf{0}, \mathbf{I}) $\)  (시작 샘플은 표준 정규분포에서 샘플링)
    
    2. **For \( t = T, \dots, 1 \) do**:
       - \($ \mathbf{z} \sim \mathcal{N}(\mathbf{0}, \mathbf{I})$ \) if \($ t > 1 $\), else \($ \mathbf{z} = 0 $\)  
         (마지막 단계가 아니면 가우시안 노이즈 추가)
    
       - 업데이트:
         $
         \mathbf{x}_{t-1} = \frac{1}{\sqrt{\alpha_t}} 
         \left( \mathbf{x}_t - \frac{1 - \alpha_t}{\sqrt{1 - \bar{\alpha}_t}} \epsilon_\theta(\mathbf{x}_t, t) \right) 
         + \sigma_t \mathbf{z}
         $
    
    3. **Return** \( $\mathbf{x}_0 $\)  (최종 생성된 샘플)

### 라이브러리

In [8]:
# torch
import torch
import torch.nn as nn
from torch.nn import init
import torch.nn.functional as F
import math

# dataset
from torchvision.datasets import CIFAR10
from torchvision import transforms

# check cuda
if torch.cuda.is_available():
    device = torch.device('cuda:0')
else:
    torch.device("cpu")
print(" Device: ", device)

 Device:  cuda:0


### 기본 변수 코드
<details>
  <summary> torch 예제 코드 </summary>
    
1. cumprod 함수 예제
    
    ``` python
    
    a = torch.tensor([5, 7, 10])
    torch.cumprod(a, dim = 0)
    # 출력: tensor([  5,  35, 350])
    
    ```

---

2. pad 함수 예제

    ```python
    a = torch.tensor([1,2,3,4])
    
    # 앞쪽에 2개, 뒤쪽에 3개 패딩 추가 , 기본 값 0 -> 9
    F.pad(a, (2, 3), value = 9)
    # 출력:: tensor([9, 9, 1, 2, 3, 4, 9, 9, 9])
    ```
</details>

In [23]:
# time t
T = 1000 

# beta: linear하게 증가
betas = torch.linspace(1e-4, 0.02, T).to(device) 

# alpha는 beta의 변형이므로
alphas = 1 - betas

# alpha bar는 alhpa의 누적 합 -> torch의 cumprod 함수
alphas_bar = torch.cumprod(alphas, dim = 0, ).to(device)

# alpha bar 의 t-1도 sampling 과정에서 필요하다. 
# 맨 처음에 alpha가 하나도 없었다는 뜻으로 맨 앞에 1을 추가.
alphas_bar_prev = F.pad(alphas_bar[:-1], (1, 0), value = 1)

# continue

### 훈련 코드

### 샘플링 코드